In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [10]:
r = requests.get("https://kakimuvee.net",headers = headers)
soup = BeautifulSoup(r.content, "html.parser")
a = soup.find('div', attrs = {"class":"header-content__container"})

In [18]:
links = [x.find('a').get('href') for x in a.find_all('li')]

In [21]:
links = [i for i in links if 'category' in i]
links

['https://kakimuvee.net/category/berita/',
 'https://kakimuvee.net/category/slot-asia/',
 'https://kakimuvee.net/category/siri-tv/',
 'https://kakimuvee.net/category/official-trailer/',
 'https://kakimuvee.net/category/review/',
 'https://kakimuvee.net/category/berita/tempatan/',
 'https://kakimuvee.net/category/streaming/',
 'https://kakimuvee.net/category/trivia/']

In [ ]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('h2', attrs = {"class":"entry-title"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('h2', attrs = {"class":"entry-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)

In [36]:
max_worker = 5


for url in links:
    
    print(url)
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    mok = soup.find('div',attrs={"class":"nav-links"})
    a = mok.find_all('a', attrs = {"class":"page-numbers"})
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()

https://kakimuvee.net/category/berita/
314


100%|██████████| 63/63 [05:09<00:00,  4.91s/it]


https://kakimuvee.net/category/slot-asia/
92


100%|██████████| 19/19 [01:33<00:00,  4.94s/it]


https://kakimuvee.net/category/siri-tv/
38


...

100%|██████████| 11/11 [00:56<00:00,  5.09s/it]


https://kakimuvee.net/category/streaming/
6


100%|██████████| 2/2 [00:08<00:00,  4.37s/it]


https://kakimuvee.net/category/trivia/
43


100%|██████████| 9/9 [00:45<00:00,  5.08s/it]


In [37]:
hrefs = list(set(hrefs))

In [39]:
len(hrefs)

8732

In [40]:
with open(f'kakimuvee-link.json', 'a') as f:
    json.dump(hrefs, f)

In [ ]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1', class_="entry-title penci-entry-title penci-title-").text
        h = soup.find('div', class_="penci-entry-content entry-content")
        p = [p.text for p in h.find_all() if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    
    return data

In [ ]:
max_workers = 3

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('kakimuvee-2.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

 98%|█████████▊| 8586/8732 [1:48:37<01:01,  2.39it/s]  